# Tutorial: Counterfactual explanations for scorecard with continuous target

This tutorial shows how to generate counterfactual explanations on scorecard models with continuous target. The dataset for this tutorial is https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_california_housing.html.

In [1]:
import pandas as pd

from sklearn.datasets import fetch_california_housing
from sklearn.linear_model import HuberRegressor

from optbinning import BinningProcess
from optbinning import Scorecard
from optbinning.scorecard import Counterfactual

Load the dataset

In [2]:
data = fetch_california_housing()

target = "target"
variable_names = data.feature_names
X = pd.DataFrame(data.data, columns=variable_names)
y = data.target

#### Scorecard model

First, we develop a scorecard following the steps presented in previous tutorials.

In [3]:
binning_process = BinningProcess(variable_names)

estimator = HuberRegressor(max_iter=200)

scorecard = Scorecard(binning_process=binning_process,
                      estimator=estimator, scaling_method="min_max",
                      scaling_method_params={"min": 0, "max": 100},
                      reverse_scorecard=True,verbose=True)

scorecard.fit(X, y)

2022-02-18 11:22:43,585 | INFO : Scorecard building process started.
2022-02-18 11:22:43,646 | INFO : Options: check parameters.
2022-02-18 11:22:43,649 | INFO : Dataset: continuous target.
2022-02-18 11:22:43,650 | INFO : Binning process started.
2022-02-18 11:22:45,629 | INFO : Binning process terminated. Time: 1.9775s
2022-02-18 11:22:45,630 | INFO : Fitting estimator.
2022-02-18 11:22:46,340 | INFO : Fitting terminated. Time 0.7096s
2022-02-18 11:22:46,342 | INFO : Scorecard table building started.
2022-02-18 11:22:46,545 | INFO : Scorecard table terminated. Time: 0.2030s
2022-02-18 11:22:46,546 | INFO : Scorecard building process terminated. Time: 2.9601s


Scorecard(binning_process=BinningProcess(variable_names=['MedInc', 'HouseAge',
                                                         'AveRooms',
                                                         'AveBedrms',
                                                         'Population',
                                                         'AveOccup', 'Latitude',
                                                         'Longitude']),
          estimator=HuberRegressor(max_iter=200), reverse_scorecard=True,
          scaling_method='min_max',
          scaling_method_params={'max': 100, 'min': 0}, verbose=True)

#### Generating counterfactual explanations

As an input data point or query, we select the first sample. Note that a query must be either a dictionary of a pandas DataFrame.

In [4]:
query = X.iloc[0, :].to_frame().T

In [5]:
query

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.02381,322.0,2.555556,37.88,-122.23


In [6]:
scorecard.predict(query)

array([4.38634097])

The predicted outcome (house value) for this query (house) is 4.38. We want to generate counterfactual explanations to find out how to increase the house value to at least 4.5.

In [7]:
cf = Counterfactual(scorecard=scorecard, verbose=True)

In [8]:
cf.fit(X)

2022-02-18 11:22:46,654 | INFO : Counterfactual fit started.
2022-02-18 11:22:46,655 | INFO : Options: check parameters.
2022-02-18 11:22:46,657 | INFO : Compute optimization problem data.
2022-02-18 11:22:46,700 | INFO : Counterfactual fit terminated. Time: 0.0458s


Counterfactual(scorecard=Scorecard(binning_process=BinningProcess(variable_names=['MedInc',
                                                                                  'HouseAge',
                                                                                  'AveRooms',
                                                                                  'AveBedrms',
                                                                                  'Population',
                                                                                  'AveOccup',
                                                                                  'Latitude',
                                                                                  'Longitude']),
                                   estimator=HuberRegressor(max_iter=200),
                                   reverse_scorecard=True,
                                   scaling_method='min_max',
                                   scaling_method

In [9]:
cf.generate(query=query, y=4.5, outcome_type="continuous", n_cf=1,
            max_changes=3, hard_constraints=["min_outcome"])

2022-02-18 11:22:46,753 | INFO : Counterfactual generation started.
2022-02-18 11:22:46,759 | INFO : Options: check parameters.
2022-02-18 11:22:46,774 | INFO : Options: check objectives and constraints.
2022-02-18 11:22:46,779 | INFO : Optimizer started.
2022-02-18 11:22:46,785 | INFO : Optimizer: build model...
2022-02-18 11:22:46,843 | INFO : Optimizer: solve...
2022-02-18 11:22:47,068 | INFO : Optimizer terminated. Time: 0.2825s
2022-02-18 11:22:47,069 | INFO : Post-processing started.
2022-02-18 11:22:47,083 | INFO : Post-processing terminated. Time: 0.0129s
2022-02-18 11:22:47,085 | INFO : Counterfactual generation terminated. Status: OPTIMAL. Time: 0.3317s


Counterfactual(scorecard=Scorecard(binning_process=BinningProcess(variable_names=['MedInc',
                                                                                  'HouseAge',
                                                                                  'AveRooms',
                                                                                  'AveBedrms',
                                                                                  'Population',
                                                                                  'AveOccup',
                                                                                  'Latitude',
                                                                                  'Longitude']),
                                   estimator=HuberRegressor(max_iter=200),
                                   reverse_scorecard=True,
                                   scaling_method='min_max',
                                   scaling_method

In [10]:
cf.information()

optbinning (Version 0.13.1)
Copyright (c) 2019-2022 Guillermo Navas-Palencia, Apache License 2.0

  Status  : OPTIMAL                         

  Solver statistics
    Type                                 mip
    Number of variables                   42
    Number of constraints                120
    Objective value                   7.6866
    Best objective bound              7.6866

  Objectives
    proximity                         0.7069
    closeness                         6.9798

  Timing
    Total time                          0.34 sec
    Fit                                 0.05 sec   ( 13.42%)
    Solver                              0.28 sec   ( 82.80%)
    Post-processing                     0.01 sec   (  4.56%)



The generate counterfactual suggest increasing the block population, reduce the average house occupancy and change the house block longitude. None of them seems doable.

In [11]:
cf.display(show_only_changes=True, show_outcome=True)

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,outcome
0,-,-,-,-,"[986.50, 1160.50)","(-inf, 1.95)",-,"[-122.16, -118.33)",4.520902


Now, let's generate several counterfactuals aiming to limit the house value to 4.0.

In [12]:
cf.generate(query=query, y=4.0, outcome_type="continuous", n_cf=3,
            max_changes=3,
            hard_constraints=["diversity_features", "max_outcome"],
            time_limit=30
           ).display(show_only_changes=True, show_outcome=True)

2022-02-18 11:22:47,150 | INFO : Counterfactual generation started.
2022-02-18 11:22:47,151 | INFO : Options: check parameters.
2022-02-18 11:22:47,157 | INFO : Options: check objectives and constraints.
2022-02-18 11:22:47,159 | INFO : Optimizer started.
2022-02-18 11:22:47,159 | INFO : Optimizer: build model...
2022-02-18 11:22:47,297 | INFO : Optimizer: solve...
2022-02-18 11:23:07,999 | INFO : Optimizer terminated. Time: 20.8401s
2022-02-18 11:23:08,001 | INFO : Post-processing started.
2022-02-18 11:23:08,036 | INFO : Post-processing terminated. Time: 0.0334s
2022-02-18 11:23:08,037 | INFO : Counterfactual generation terminated. Status: OPTIMAL. Time: 20.8866s


,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,outcome
0,-,-,-,-,"[986.50, 1160.50)","[3.11, 3.24)",-,"[-122.16, -118.33)",3.968866
0,"[5.79, 6.82)",-,-,"(-inf, 0.95)",-,-,-,"[-122.16, -118.33)",3.227336
0,"[5.79, 6.82)",-,-,-,"[986.50, 1160.50)",-,-,"[-122.16, -118.33)",3.158199


And the same generation enforcing diversity on feature values.

In [13]:
cf.generate(query=query, y=3.0, outcome_type="continuous", n_cf=3,
            max_changes=3,
            hard_constraints=["diversity_features", "diversity_values", "max_outcome"],
            time_limit=30
           ).display(show_only_changes=True, show_outcome=True)

2022-02-18 11:23:08,076 | INFO : Counterfactual generation started.
2022-02-18 11:23:08,079 | INFO : Options: check parameters.
2022-02-18 11:23:08,085 | INFO : Options: check objectives and constraints.
2022-02-18 11:23:08,086 | INFO : Optimizer started.
2022-02-18 11:23:08,087 | INFO : Optimizer: build model...
2022-02-18 11:23:08,221 | INFO : Optimizer: solve...
2022-02-18 11:23:32,984 | INFO : Optimizer terminated. Time: 24.8974s
2022-02-18 11:23:32,986 | INFO : Post-processing started.
2022-02-18 11:23:33,016 | INFO : Post-processing terminated. Time: 0.0294s
2022-02-18 11:23:33,017 | INFO : Counterfactual generation terminated. Status: OPTIMAL. Time: 24.9414s


,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,outcome
0,"[5.79, 6.82)",-,-,-,-,"[3.11, 3.24)",-,"[-118.16, inf)",2.981844
0,"[4.53, 5.04)",-,"[6.12, 6.37)",-,-,-,-,"[-118.33, -118.26)",2.486897
0,"[5.04, 5.79)",-,-,-,"[986.50, 1160.50)",-,-,"[-122.16, -118.33)",2.646881
